In [165]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.stattools import coint


In [166]:
# Get the data from tsm and qcom with inteval of 30 minutes

tsm = yf.Ticker("TSM")
qcom = yf.Ticker("QCOM")

tsm_data = tsm.history(period="60d", interval="30m")
qcom_data = qcom.history(period="60d", interval="30m")

tsm_data.reset_index(inplace=True)
qcom_data.reset_index(inplace=True)

if ('Date' in tsm_data.columns) and ('Date' in qcom_data.columns):
    print("Date column exists")
    tsm_data.rename(columns={"Date": "Datetime"}, inplace=True)
    qcom_data.rename(columns={"Date": "Datetime"}, inplace=True)

tsm_data.set_index("Datetime", inplace=True)
qcom_data.set_index("Datetime", inplace=True)

In [167]:
# plot both tickers on go figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=qcom_data.index, y=qcom_data['Close'], name='QCOM'))

fig.show()

In [168]:
print(tsm_data)

                                 Open        High         Low       Close  \
Datetime                                                                    
2024-03-12 09:30:00-04:00  144.520004  144.929993  140.570007  143.350006   
2024-03-12 10:00:00-04:00  143.399994  143.949997  142.479996  143.410004   
2024-03-12 10:30:00-04:00  143.429993  145.759903  143.389999  144.869995   
2024-03-12 11:00:00-04:00  144.880005  146.240005  144.627594  146.190002   
2024-03-12 11:30:00-04:00  146.179993  146.699997  145.199997  145.389999   
...                               ...         ...         ...         ...   
2024-06-05 13:30:00-04:00  162.005005  162.250000  161.550003  162.190002   
2024-06-05 14:00:00-04:00  162.229996  162.669998  162.110199  162.630005   
2024-06-05 14:30:00-04:00  162.634995  163.119904  162.392807  163.080002   
2024-06-05 15:00:00-04:00  163.070007  163.289993  162.787994  162.960007   
2024-06-05 15:30:00-04:00  162.955002  163.139999  161.660004  162.889999   

In [169]:
# plot eith candle stick

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

# plot moving average of 20 ticks and 80 ticks

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()

In [170]:
# count the number of times the 5 ticks MA crosses the 20 ticks MA

tsm_data['5T_MA'] = tsm_data['Close'].rolling(window=5).mean()
tsm_data['20T_MA'] = tsm_data['Close'].rolling(window=20).mean()

tsm_data['signal'] = np.where(tsm_data['5T_MA'] > tsm_data['20T_MA'], 1, 0)
tsm_data['signal'] = tsm_data['signal'].diff()


print(tsm_data['signal'].value_counts())

# plot the buy and sell signals

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = tsm_data[tsm_data['signal'] == 1]
sell_signals = tsm_data[tsm_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))

fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

signal
 0.0    730
 1.0     25
-1.0     24
Name: count, dtype: int64


In [171]:
# calculate profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal'] == 1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal'] == -1, tsm_data['Close'], np.nan)

tsm_data['buy_price'] = tsm_data['buy_price'].ffill()
tsm_data['sell_price'] = tsm_data['sell_price'].ffill()

tsm_data['PnL'] = tsm_data['sell_price'] - tsm_data['buy_price']

print(tsm_data['PnL'].sum())

# plot the profit and loss

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = tsm_data[tsm_data['signal'] == 1]
sell_signals = tsm_data[tsm_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

-623.3622894287109


In [172]:
ratio_0 = (tsm_data['Close'].iloc[0]) / (qcom_data['Close'].iloc[0])
ratio_0

0.8313036654718491

In [173]:
qcom_data['Close_NORM'] = qcom_data['Close'].multiply(ratio_0).round(6)

In [174]:
tsm_qcom_dif = tsm_data['Close'] - qcom_data['Close_NORM']
tsm_qcom_dif

Datetime
2024-03-12 09:30:00-04:00    1.035156e-07
2024-03-12 10:00:00-04:00   -4.807634e-02
2024-03-12 10:30:00-04:00    7.801291e-01
2024-03-12 11:00:00-04:00    2.499158e+00
2024-03-12 11:30:00-04:00    2.089869e+00
                                 ...     
2024-06-05 13:30:00-04:00   -1.390505e+01
2024-06-05 14:00:00-04:00   -1.353987e+01
2024-06-05 14:30:00-04:00   -1.337667e+01
2024-06-05 15:00:00-04:00   -1.289397e+01
2024-06-05 15:30:00-04:00   -1.330481e+01
Length: 780, dtype: float64

In [175]:
# Plot the difference between the two tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif, name='TSM-QCOM'))

fig.show()

In [176]:
# Plot the two normalized tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=qcom_data.index, y=qcom_data['Close_NORM'], name='QCOM'))

fig.show()

In [177]:
# Plot the difference and its moving average

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif, name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()


In [178]:
tsm_qcom_dif = tsm_qcom_dif.to_frame()
tsm_qcom_dif.columns = ['Close']

tsm_qcom_dif['index'] = np.arange(tsm_qcom_dif.shape[0])


In [179]:
tsm_qcom_dif['10T_MA'] = tsm_qcom_dif['Close'].rolling(window=10).mean()
tsm_qcom_dif['60T_MA'] = tsm_qcom_dif['Close'].rolling(window=60).mean()

tsm_qcom_dif['signal'] = np.where(tsm_qcom_dif['10T_MA'] > tsm_qcom_dif['60T_MA'], -1, 0)
tsm_qcom_dif['signal'] = tsm_qcom_dif['signal'].diff()

print(tsm_qcom_dif['signal'].value_counts())

# Plot the signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif['Close'], name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=10).mean()['Close'], name='10 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=60).mean()['Close'], name='60 Ticks MA'))

buy_signals = tsm_qcom_dif[tsm_qcom_dif['signal'] == 1]
#last_sell_time = tsm_qcom_dif[tsm_qcom_dif['signal'] == -1][-1]['Datetime']
#print(tsm_qcom_dif[tsm_qcom_dif['signal'] == -1][-1]['Datetime'])
sell_signals = tsm_qcom_dif[tsm_qcom_dif['signal'] == -1][:-1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

 # -1 buy || 1 sell

tsm_data['signal'] = tsm_qcom_dif['signal']
qcom_data['signal'] = -tsm_qcom_dif['signal'] # opposite signal

fig.update_layout(title='TSM-QCOM Moving Average Crossover Strategy (Short Average = 10, Long Average = 60) (60 days)')

fig.show()

# plot the profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal'] == 1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal'] == -1, tsm_data['Close'], np.nan)

qcom_data['buy_price'] = np.where(qcom_data['signal'] == 1, qcom_data['Close_NORM'], np.nan)
qcom_data['sell_price'] = np.where(qcom_data['signal'] == -1, qcom_data['Close_NORM'], np.nan)

tsm_profit = tsm_data['sell_price'].sum() - tsm_data['buy_price'].sum()
qcom_profit = qcom_data['sell_price'].sum() - qcom_data['buy_price'].sum()

print('TSMC Profit => ', tsm_profit)
print('QCOM Profit => ', qcom_profit)

pair_tsm_qcom_profit = tsm_profit + qcom_profit

print('Pair profit => ', pair_tsm_qcom_profit)

fig.write_image('tsm_qcom_macd.png', width=1100, height=350)


signal
 0.0    758
-1.0     11
 1.0     10
Name: count, dtype: int64


TSMC Profit =>  143.35829162597656
QCOM Profit =>  -150.35905700000012
Pair profit =>  -7.000765374023558


In [180]:
#Consider TSM as the first and QCOM as the second ticker.
#When there is a buy signal for TSM, we buy TSM and sell QCOM.
#When there is a sell signal for TSM, we sell TSM and buy QCOM.

In [181]:
#Copy signal column from difference dataframe to TSM dataframe
tsm_data['signal'] = tsm_qcom_dif['signal']
qcom_data['signal'] = -tsm_qcom_dif['signal']

In [182]:
tsm_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,5T_MA,20T_MA,signal,buy_price,sell_price,PnL
Datetime,,,,,,,,,,,,,
2024-03-12 09:30:00-04:00,144.520004,144.929993,140.570007,143.350006,4309889,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2024-03-12 10:00:00-04:00,143.399994,143.949997,142.479996,143.410004,1718323,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2024-03-12 10:30:00-04:00,143.429993,145.759903,143.389999,144.869995,2176716,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2024-03-12 11:00:00-04:00,144.880005,146.240005,144.627594,146.190002,1452718,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2024-03-12 11:30:00-04:00,146.179993,146.699997,145.199997,145.389999,1080245,0.0,0.0,144.642001,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-05 13:30:00-04:00,162.005005,162.250000,161.550003,162.190002,633040,0.0,0.0,162.666901,156.591732,0.0,NaN,NaN,-9.139999
2024-06-05 14:00:00-04:00,162.229996,162.669998,162.110199,162.630005,633043,0.0,0.0,162.612903,157.086982,0.0,NaN,NaN,-9.139999
2024-06-05 14:30:00-04:00,162.634995,163.119904,162.392807,163.080002,481148,0.0,0.0,162.550003,157.660982,0.0,NaN,NaN,-9.139999


In [183]:
tsm_data.reset_index(inplace=True)
qcom_data.reset_index(inplace=True)

# Convert datetime columns to indexes
tsm_data.set_index('Datetime', inplace=True)
qcom_data.set_index('Datetime', inplace=True)

# Find common datetime values
common_dates = tsm_data.index.intersection(qcom_data.index)

# Align the data and handle missing values
tsm_close_aligned = tsm_data.loc[common_dates, 'Close']
qcom_close_aligned = qcom_data.loc[common_dates, 'Close_NORM']

# Perform cointegration test
cointegration_test_result = coint(tsm_close_aligned, qcom_close_aligned)


# Print the result
print("Cointegration Test Result:")
print("Test statistic:", cointegration_test_result[0])
print("p-value:", cointegration_test_result[1])
print("Critical values:", cointegration_test_result[2])



Cointegration Test Result:
Test statistic: -3.1917366260784528
p-value: 0.0713206154139108
Critical values: [-3.91055417 -3.34398476 -3.0498989 ]


In [184]:
# Calculate the z score of the difference
tsm_qcom_dif['z_score'] = (tsm_qcom_dif['Close'] - tsm_qcom_dif['Close'].rolling(window=20).mean()) / tsm_qcom_dif['Close'].rolling(window=20).std()

In [185]:
# function to implement a flag that is (-1 sell order open, 0 no order open, 1 buy order open) to control the signal_z_score signals

def restrict_signals(dataset, signal_col, flag_col):
    dataset[flag_col] = 0
    # cicle to put flags
    for i in range(dataset.shape[0]):
        if dataset[signal_col].iloc[i] == 1:
            if dataset[flag_col].iloc[i] < 1:
                dataset[flag_col].iloc[i] += 1
                print('Buy signal')
            else:
                dataset[signal_col].iloc[i] = 0
                print('Reset buy signal')
        elif dataset[signal_col].iloc[i] == -1:
            if dataset[flag_col].iloc[i] > -1:
                dataset[flag_col].iloc[i] += -1
                print('Sell signal')
            else:
                dataset[signal_col].iloc[i] = 0
                print('Reset sell signal')

    return dataset
        

In [186]:
tsm_qcom_dif.reset_index(inplace=True)

In [187]:
tsm_qcom_dif


,Datetime,Close,index,10T_MA,60T_MA,signal,z_score
0,2024-03-12 09:30:00-04:00,1.035156e-07,0,NaN,NaN,NaN,NaN
1,2024-03-12 10:00:00-04:00,-4.807634e-02,1,NaN,NaN,0.0,NaN
2,2024-03-12 10:30:00-04:00,7.801291e-01,2,NaN,NaN,0.0,NaN
3,2024-03-12 11:00:00-04:00,2.499158e+00,3,NaN,NaN,0.0,NaN
4,2024-03-12 11:30:00-04:00,2.089869e+00,4,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...
775,2024-06-05 13:30:00-04:00,-1.390505e+01,775,-13.317356,-16.634877,0.0,0.597146
776,2024-06-05 14:00:00-04:00,-1.353987e+01,776,-12.922990,-16.557340,0.0,0.687262
777,2024-06-05 14:30:00-04:00,-1.337667e+01,777,-13.007796,-16.484835,0.0,0.678740
778,2024-06-05 15:00:00-04:00,-1.289397e+01,778,-13.080164,-16.409101,0.0,0.801440


In [188]:
# plot the z score along with the buy and sell signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif['Datetime'], y=tsm_qcom_dif['z_score'], name='Z Score'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif['Datetime'], y=[2] * len(tsm_qcom_dif), name='Upper Threshold'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif['Datetime'], y=[-2] * len(tsm_qcom_dif), name='Lower Threshold'))

#buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['z_score'] < -2]
#sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['z_score'] > 2]

# TODO: check this
#tsm_qcom_dif['signal_z_score'] = np.where(((tsm_qcom_dif['z_score'] < -2) and (tsm_qcom_dif['z_score_sig_flag'] < 1)), 1, 0) + np.where(((tsm_qcom_dif['z_score'] > 2) and (tsm_qcom_dif['z_score_sig_flag'] > 1)), -1, 0)
tsm_qcom_dif['signal_z_score'] = np.where((tsm_qcom_dif['z_score'] < -2), 1, 0) + np.where((tsm_qcom_dif['z_score'] > 2), -1, 0)

#tsm_qcom_dif = restrict_signals(tsm_qcom_dif, 'signal_z_score', 'z_score_sig_flag')

""" INIT """
flag_signal_z_score = 0

for i in range(tsm_qcom_dif.shape[0]):
    print(tsm_qcom_dif['signal_z_score'].iloc[i])
    if tsm_qcom_dif['signal_z_score'].iloc[i] == 1:
        print('in', flag_signal_z_score)
        if flag_signal_z_score < 1:
            flag_signal_z_score = 1
            print('Buy signal')
        else:
            tsm_qcom_dif['signal_z_score'].iloc[i] = 0
            print('Reset buy signal')
    elif tsm_qcom_dif['signal_z_score'].iloc[i] == -1:
        if flag_signal_z_score > -1:
            flag_signal_z_score = -1
            print('Sell signal')
        else:
            tsm_qcom_dif['signal_z_score'].iloc[i] = 0
            print('Reset sell signal')
    print(tsm_qcom_dif['signal_z_score'].iloc[i])
    

""" END """
# plot the signals
buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1]
sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1]

fig.add_trace(go.Scatter(x=buy_signals_z_score['Datetime'], y=buy_signals_z_score['z_score'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals_z_score['Datetime'], y=sell_signals_z_score['z_score'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.update_layout(title='Z Score with Buy/Sell Signals (60 days)')

fig.show()


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in 0
Buy signal
1
1
in 1
Reset buy signal
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
1
in 1
Reset buy signal
0
1
in 1
Reset buy signal
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
Sell signal
-1
-1
Reset sell signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in -1
Buy signal
1
1
in 1
Reset buy signal
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
Sell signal
-1
-1
Reset sell signal
0
-1
Reset sell signa

/home/alexandre/.local/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/tmp/ipykernel_164028/3528568253.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_164028/3528568253.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/alexandre/.local/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of D

### Try to remove times without data

In [189]:
#tsm_qcom_dif.set_index('Datetime', inplace=True)

# Add column with index to the dataframe
tsm_qcom_dif['index'] = np.arange(tsm_qcom_dif.shape[0])

In [190]:
tsm_qcom_dif

,Datetime,Close,index,10T_MA,60T_MA,signal,z_score,signal_z_score
0,2024-03-12 09:30:00-04:00,1.035156e-07,0,NaN,NaN,NaN,NaN,0
1,2024-03-12 10:00:00-04:00,-4.807634e-02,1,NaN,NaN,0.0,NaN,0
2,2024-03-12 10:30:00-04:00,7.801291e-01,2,NaN,NaN,0.0,NaN,0
3,2024-03-12 11:00:00-04:00,2.499158e+00,3,NaN,NaN,0.0,NaN,0
4,2024-03-12 11:30:00-04:00,2.089869e+00,4,NaN,NaN,0.0,NaN,0
...,...,...,...,...,...,...,...,...
775,2024-06-05 13:30:00-04:00,-1.390505e+01,775,-13.317356,-16.634877,0.0,0.597146,0
776,2024-06-05 14:00:00-04:00,-1.353987e+01,776,-12.922990,-16.557340,0.0,0.687262,0
777,2024-06-05 14:30:00-04:00,-1.337667e+01,777,-13.007796,-16.484835,0.0,0.678740,0
778,2024-06-05 15:00:00-04:00,-1.289397e+01,778,-13.080164,-16.409101,0.0,0.801440,0


In [191]:
# plot the z score along with the buy and sell signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif['index'], y=tsm_qcom_dif['z_score'], name='Z Score'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif['index'], y=[2] * len(tsm_qcom_dif), name='Upper Threshold'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif['index'], y=[-2] * len(tsm_qcom_dif), name='Lower Threshold'))

#buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['z_score'] < -2]
#sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['z_score'] > 2]

# TODO: check this
#tsm_qcom_dif['signal_z_score'] = np.where(((tsm_qcom_dif['z_score'] < -2) and (tsm_qcom_dif['z_score_sig_flag'] < 1)), 1, 0) + np.where(((tsm_qcom_dif['z_score'] > 2) and (tsm_qcom_dif['z_score_sig_flag'] > 1)), -1, 0)
tsm_qcom_dif['signal_z_score'] = np.where((tsm_qcom_dif['z_score'] < -2), 1, 0) + np.where((tsm_qcom_dif['z_score'] > 2), -1, 0)

#tsm_qcom_dif = restrict_signals(tsm_qcom_dif, 'signal_z_score', 'z_score_sig_flag')

""" INIT """
flag_signal_z_score = 0

for i in range(tsm_qcom_dif.shape[0]):
    print(tsm_qcom_dif['signal_z_score'].iloc[i])
    if tsm_qcom_dif['signal_z_score'].iloc[i] == 1:
        print('in', flag_signal_z_score)
        if flag_signal_z_score < 1:
            flag_signal_z_score = 1
            print('Buy signal')
        else:
            tsm_qcom_dif['signal_z_score'].iloc[i] = 0
            print('Reset buy signal')
    elif tsm_qcom_dif['signal_z_score'].iloc[i] == -1:
        if flag_signal_z_score > -1:
            flag_signal_z_score = -1
            print('Sell signal')
        else:
            tsm_qcom_dif['signal_z_score'].iloc[i] = 0
            print('Reset sell signal')
    print(tsm_qcom_dif['signal_z_score'].iloc[i])
    

""" END """
# plot the signals
buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1]
sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1]

fig.add_trace(go.Scatter(x=buy_signals_z_score['index'], y=buy_signals_z_score['z_score'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals_z_score['index'], y=sell_signals_z_score['z_score'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.update_layout(title='Z Score with Buy/Sell Signals (60 days)')

fig.show()


/tmp/ipykernel_164028/598224679.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_164028/598224679.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in 0
Buy signal
1
1
in 1
Reset buy signal
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
1
in 1
Reset buy signal
0
1
in 1
Reset buy signal
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
Sell signal
-1
-1
Reset sell signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
in -1
Buy signal
1
1
in 1
Reset buy signal
0
1
in 1
Reset buy signal
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
Sell signal
-1
-1
Reset sell signal
0
-1
Reset sell signa

In [192]:
# save fig to png
fig.write_image("tsm_qcom_z_score.png", width=1000, height=350)

### Check if the number of sell orders is equal to buy orders and remove the last one in case of odd number of orders.


In [193]:

count_buys = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1].shape[0]
count_sells = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1].shape[0]

print('Count buys => ', count_buys)
print('Count sells => ', count_sells)

dif_buys_sells = count_buys - count_sells
print('Dif buys/sells => ', dif_buys_sells)

tsm_qcom_dif.reset_index(inplace=True)

if dif_buys_sells > 0:
    last_buy_signal = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1].iloc[[-1]]['Datetime']
    #print(last_buy_signal)
    if not last_buy_signal.empty:
        print('Datetime of signal to set to 0 => ', str(last_buy_signal.iloc[0]))
        
        tsm_qcom_dif.loc[tsm_qcom_dif['Datetime'] == str(last_buy_signal.iloc[0]), 'signal_z_score'] = 0
        #print(tsm_qcom_dif.loc[tsm_qcom_dif['Datetime'] == str(last_sell_signal.iloc[0])])
        print('Delete last BUY signal')

        #print('All buy signals')
        #print(tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1])


elif dif_buys_sells < 0:
    last_sell_signal = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1].iloc[[-1]]['Datetime']
    #print(last_sell_signal)
    if not last_sell_signal.empty:
        print('Datetime of signal to set to 0 => ', str(last_sell_signal.iloc[0]))
        
        tsm_qcom_dif.loc[tsm_qcom_dif['Datetime'] == str(last_sell_signal.iloc[0]), 'signal_z_score'] = 0
        #print(tsm_qcom_dif.loc[tsm_qcom_dif['Datetime'] == str(last_sell_signal.iloc[0])])
        print('Delete last SELL signal')

        #print('All sell signals')
        #print(tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1])



Count buys =>  7
Count sells =>  7
Dif buys/sells =>  0


## Calculate Profit using Z-score

In [194]:
# Plot the signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif['index'], y=tsm_qcom_dif['Close'], name='TSM-QCOM'))
#fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=10).mean()['Close'], name='10 Ticks MA'))
#fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=50).mean()['Close'], name='50 Ticks MA'))


buy_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == 1]
sell_signals_z_score = tsm_qcom_dif[tsm_qcom_dif['signal_z_score'] == -1]

fig.add_trace(go.Scatter(x=buy_signals_z_score['index'], y=buy_signals_z_score['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals_z_score['index'], y=sell_signals_z_score['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.update_layout(title='TSM-QCOM Buy/Sell orders using Z-Score (60 days)')

# 1 buy || - 1 sell

tsm_data.reset_index(inplace=True)
qcom_data.reset_index(inplace=True)

tsm_data['signal_z_score'] = tsm_qcom_dif['signal_z_score']
qcom_data['signal_z_score'] = - tsm_qcom_dif['signal_z_score'] # opposite signal

fig.show()

# plot the profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal_z_score'] == 1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal_z_score'] == -1, tsm_data['Close'], np.nan)

qcom_data['buy_price'] = np.where(qcom_data['signal_z_score'] == 1, qcom_data['Close_NORM'], np.nan)
qcom_data['sell_price'] = np.where(qcom_data['signal_z_score'] == -1, qcom_data['Close_NORM'], np.nan)

tsm_profit = tsm_data['sell_price'].sum() - tsm_data['buy_price'].sum()
qcom_profit = qcom_data['sell_price'].sum() - qcom_data['buy_price'].sum()

print('TSMC Profit => ', tsm_profit)
print('QCOM Profit => ', qcom_profit)

pair_tsm_qcom_profit = tsm_profit + qcom_profit

print('Pair profit => ', pair_tsm_qcom_profit)


TSMC Profit =>  23.18560791015625
QCOM Profit =>  -19.969908000000146
Pair profit =>  3.215699910156104


In [195]:
fig.write_image("tsm_qcom_spread_using_z_score.png", width=1000, height=350)


In [196]:
#Testar Métodos de Mercados Diferentes, Mercados Tecnológicos
# A cointegração muda com 